In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from geostacks import SpatialIndexLS8

In [10]:
ls8_index = SpatialIndexLS8('./LS8_cornerPts.xlsx')
ls8_index.read()
# ls8_index.footprint

In [11]:
query_pt = [-50., 69.]
idx = ls8_index.query_pathrow(query_pt)
ls8_index.footprint.loc[idx]

,path,row,lat_CTR,lon_CTR,geometry
1747,8,12,68.279631,-49.545762,"POLYGON ((-50.91000 69.35600, -46.41400 68.660..."
1994,9,11,69.606373,-49.402408,"POLYGON ((-50.77700 70.69500, -46.05000 69.955..."
2242,10,11,69.606373,-50.947472,"POLYGON ((-52.32200 70.69500, -47.59500 69.955..."
20071,81,233,69.606373,-48.123782,"POLYGON ((-44.88600 69.19800, -49.35900 68.507..."
20319,82,233,69.606373,-49.668847,"POLYGON ((-46.43100 69.19800, -50.90400 68.507..."
20566,83,232,68.279631,-49.525493,"POLYGON ((-46.50100 67.84400, -50.76100 67.192..."
20567,83,233,69.606373,-51.213911,"POLYGON ((-47.97600 69.19800, -52.44900 68.507..."
20814,84,232,68.279631,-51.070557,"POLYGON ((-48.04600 67.84400, -52.30600 67.192..."
